<h2>Importando bibliotecas</h2>

In [1]:
import findspark
findspark.init()
import pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
import pandas as pd
import pyspark.pandas as ps

<h2> Criando sessão do spark + configurações de conexão com bucket </h2>

In [4]:
spark = SparkSession.builder.appName('Steam API - Tratamento').getOrCreate()

22/03/15 21:18:22 WARN Utils: Your hostname, moon resolves to a loopback address: 127.0.1.1; using 192.168.0.185 instead (on interface wlo1)
22/03/15 21:18:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/15 21:18:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.EnvironmentVariableCredentialsProvider')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.path.style.access', 'true')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://localhost:9000')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.connection.ssl.enabled', 'false')
    
load_config(spark.sparkContext)


<h2> Lendo dados do bucket </h2>

In [6]:
df = spark.read.json('s3a://bronze/topics/steam/*', multiLine=True)

22/03/15 21:18:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
df.printSchema()

root
 |-- appid: string (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- language: string (nullable = true)
 |-- last_played: long (nullable = true)
 |-- num_games_owned: long (nullable = true)
 |-- num_reviews: long (nullable = true)
 |-- playtime_forever: long (nullable = true)
 |-- playtime_last_two_weeks: long (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- recommendationid: string (nullable = true)
 |-- review: string (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- steamid: string (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- voted_up: boolean (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- votes_up: long (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)



In [8]:
df.count()

87

In [9]:
#df = df.to_pandas_on_spark()

In [10]:
df.show(1, truncate=True)

+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
| appid|comment_count|language|last_played|num_games_owned|num_reviews|playtime_forever|playtime_last_two_weeks|received_for_free|recommendationid|review|steam_purchase|          steamid|timestamp_created|timestamp_updated|voted_up|votes_funny|votes_up|weighted_vote_score|written_during_early_access|
+------+-------------+--------+-----------+---------------+-----------+----------------+-----------------------+-----------------+----------------+------+--------------+-----------------+-----------------+-----------------+--------+-----------+--------+-------------------+---------------------------+
|271590|            0| english| 1647381998|             47|          6|           33675|      

### Convertendo de timestamp UNIX para datetime

In [11]:
from pyspark.sql import functions as f
from pyspark.sql import types as t
from datetime import datetime

In [12]:
df.withColumn('last_played', f.date_format(df.last_played.cast(dataType=t.TimestampType()), "yyyy-MM-dd")) \
  .withColumn('timestamp_created', f.date_format(df.timestamp_created.cast(dataType=t.TimestampType()), "yyyy-MM-dd")) \
  .withColumn('timestamp_updated', f.date_format(df.timestamp_updated.cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

DataFrame[appid: string, comment_count: bigint, language: string, last_played: string, num_games_owned: bigint, num_reviews: bigint, playtime_forever: bigint, playtime_last_two_weeks: bigint, received_for_free: boolean, recommendationid: string, review: string, steam_purchase: boolean, steamid: string, timestamp_created: string, timestamp_updated: string, voted_up: boolean, votes_funny: bigint, votes_up: bigint, weighted_vote_score: string, written_during_early_access: boolean]

In [13]:
df2 = df.withColumn('last_played', f.to_date(df.last_played.cast(dataType=t.TimestampType()))) \
        .withColumn('timestamp_created', f.to_date(df.timestamp_created.cast(dataType=t.TimestampType()))) \
        .withColumn('timestamp_updated', f.to_date(df.timestamp_updated.cast(dataType=t.TimestampType())))

In [14]:
df2 = df2.withColumn("last_played",f.to_timestamp(df2['last_played'])) \
         .withColumn("timestamp_created",f.to_timestamp(df2['timestamp_created'])) \
         .withColumn("timestamp_updated",f.to_timestamp(df2['timestamp_updated']))

    

### Removendo possíveis registros duplicados

In [15]:
df3 = df2.drop_duplicates()

### Filtrando colunas de interesse

In [16]:
df4 = df3.select('appid', 'recommendationid', 'steamid', 'language', 'last_played', 'num_games_owned', 'playtime_forever', 'review', 'voted_up', 'votes_up','timestamp_created')

In [17]:
df4.show()

+------+----------------+-----------------+--------+-------------------+---------------+----------------+--------------------+--------+--------+-------------------+
| appid|recommendationid|          steamid|language|        last_played|num_games_owned|playtime_forever|              review|voted_up|votes_up|  timestamp_created|
+------+----------------+-----------------+--------+-------------------+---------------+----------------+--------------------+--------+--------+-------------------+
|271590|       112239162|76561198254536142| english|2022-03-15 00:00:00|             96|             920|Man, even after T...|   false|       3|2022-03-15 00:00:00|
|271590|       112253643|76561198307394253| english|2022-03-14 00:00:00|             20|            7868|Rockstar doesnt c...|   false|       0|2022-03-15 00:00:00|
|271590|       112240330|76561198449180366| english|2022-03-15 00:00:00|             27|            2535|                isok|    true|       0|2022-03-15 00:00:00|
|271590|  

## Salvando dados tratados no Silver

In [18]:
df4.write.partitionBy('appid').mode('overwrite').parquet('s3a://silver/steam_reviews/reviews.parquet')